In [53]:
from splinter import Browser
from bs4 import BeautifulSoup as soup
from webdriver_manager.chrome import ChromeDriverManager
import re
import pandas as pd
import time

# Set up Splinter
executable_path = {'executable_path': ChromeDriverManager().install()}
browser = Browser('chrome', **executable_path, headless=False)

# Visit the Mars Facts site
url = 'https://www.goodreads.com/list/show/1.Best_Books_Ever'
browser.visit(url)

html = browser.html
html_soup = soup(html, 'html.parser')

In [54]:
def GoodScrapes(pages, count=1, best_books={}):
    while count < pages:
        html = browser.html
        html_soup = soup(html, 'html.parser')
        close_popup = 'div[class="modal__close"]'
        try:
            browser.find_by_tag(close_popup).button.click()
        except:
            pass
        table = html_soup.find('table', class_='tableList')
        rows = table.find_all('tr')
        for row in rows:
            cols = row.find_all('td')
            num = int(cols[0].text)
            info = cols[2]
            name = info.a.text.strip()
            goodreads_link = 'https://www.goodreads.com' + info.findNext(class_='bookTitle')['href']
            goodreads_get_copy = goodreads_link.replace('show/', '') + '/get_a_copy'
            auth = info.findNext(class_='authorName').text
            stars = info.findNext(class_='minirating').text
            rating = re.findall(r'\d+\.\d+', stars)[0]
            best_books[num] = [name, auth, float(rating), goodreads_link, goodreads_get_copy]
        tg = 'a[class="next_page"]'
        try:
            browser.find_by_tag(tg).click()
        except:
            browser.find_by_tag(close_popup).button.click()
            browser.find_by_tag(tg).click()
        count += 1
    return best_books

def getAmazonLinks(df):
    for index, row in df.iterrows():
        url = row['get_copy']
        browser.visit(url)
        try:
            amazon_link = browser.find_by_text('Amazon')['href']
        except:
            amazon_link = 'none'
        df.loc[index, 'amazon_link'] = amazon_link
        time.sleep(0.5)

In [ ]:
# before running the next cells, sign in to goodreads so their stupid popup doesnt ruin the scraping

In [55]:
url = 'https://www.goodreads.com/list/show/1.Best_Books_Ever'
browser.visit(url)

In [12]:
# best_books = {}
# html = browser.html
# html_soup = soup(html, 'html.parser')
# close_popup = 'div[class="modal__close"]'
# try:
#     browser.find_by_tag(close_popup).button.click()
# except:
#     pass
# table = html_soup.find('table', class_='tableList')
# rows = table.find_all('tr')
# for row in rows:
#     cols = row.find_all('td')
#     num = int(cols[0].text)
#     info = cols[2]
#     name = info.a.text.strip()
#     goodreads_link = 'https://www.goodreads.com' + info.findNext(class_='bookTitle')['href']
#     goodreads_get_copy = goodreads_link.replace('show/', '') + '/get_a_copy'
#     auth = info.findNext(class_='authorName').text
#     stars = info.findNext(class_='minirating').text
#     rating = re.findall(r'\d+\.\d+', stars)[0]
#     best_books[num] = [name, auth, float(rating), goodreads_link, goodreads_get_copy]

In [56]:
best_books = GoodScrapes(3)

In [57]:
best_books

{1: ['The Hunger Games (The Hunger Games, #1)',
  'Suzanne Collins',
  4.33,
  'https://www.goodreads.com/book/show/2767052-the-hunger-games',
  'https://www.goodreads.com/book/2767052-the-hunger-games/get_a_copy'],
 2: ['Harry Potter and the Order of the Phoenix (Harry Potter, #5)',
  'J.K. Rowling',
  4.5,
  'https://www.goodreads.com/book/show/2.Harry_Potter_and_the_Order_of_the_Phoenix',
  'https://www.goodreads.com/book/2.Harry_Potter_and_the_Order_of_the_Phoenix/get_a_copy'],
 3: ['Pride and Prejudice',
  'Jane Austen',
  4.28,
  'https://www.goodreads.com/book/show/1885.Pride_and_Prejudice',
  'https://www.goodreads.com/book/1885.Pride_and_Prejudice/get_a_copy'],
 4: ['To Kill a Mockingbird',
  'Harper Lee',
  4.27,
  'https://www.goodreads.com/book/show/2657.To_Kill_a_Mockingbird',
  'https://www.goodreads.com/book/2657.To_Kill_a_Mockingbird/get_a_copy'],
 5: ['The Book Thief',
  'Markus Zusak',
  4.39,
  'https://www.goodreads.com/book/show/19063.The_Book_Thief',
  'https://ww

In [58]:
books_df = pd.DataFrame.from_dict(best_books, orient='index', columns=['name', 'author', 'stars', 'goodreads_link', 'get_copy'])
books_df.head()

,name,author,stars,goodreads_link,get_copy
1,"The Hunger Games (The Hunger Games, #1)",Suzanne Collins,4.33,https://www.goodreads.com/book/show/2767052-th...,https://www.goodreads.com/book/2767052-the-hun...
2,Harry Potter and the Order of the Phoenix (Har...,J.K. Rowling,4.50,https://www.goodreads.com/book/show/2.Harry_Po...,https://www.goodreads.com/book/2.Harry_Potter_...
3,Pride and Prejudice,Jane Austen,4.28,https://www.goodreads.com/book/show/1885.Pride...,https://www.goodreads.com/book/1885.Pride_and_...
4,To Kill a Mockingbird,Harper Lee,4.27,https://www.goodreads.com/book/show/2657.To_Ki...,https://www.goodreads.com/book/2657.To_Kill_a_...
5,The Book Thief,Markus Zusak,4.39,https://www.goodreads.com/book/show/19063.The_...,https://www.goodreads.com/book/19063.The_Book_...


In [59]:
getAmazonLinks(books_df)

In [36]:
# for index, row in books_df.iterrows():
#     url = row['get_copy']
#     browser.visit(url)
#     amazon_link = browser.find_by_text('Amazon')['href']
#     books_df.loc[index, 'amazon_link'] = amazon_link
#     time.sleep(1)
    

InvalidArgumentException: Message: invalid argument
  (Session info: chrome=109.0.5414.120)


In [60]:
books_df.to_csv('goodreads_best_books.csv')

In [52]:
browser.quit()